In [1]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [2]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [3]:
start = 1692978196
end = 1693513472

In [4]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = expTime.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../experiments.csv')
experiments.head(5)

/tmp/ipykernel_13350/1055150111.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_13350/1055150111.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,150,2023-08-25 15:43:16.3243323,2023-08-25 16:13:16.3255309,unl,reference,8,6,12,4,8,2,0.25,1.0,1692978196,1692979996
1,151,2023-08-25 16:16:49.6961810,2023-08-25 16:46:49.6974978,unl,reference,8,6,12,4,8,2,0.25,1.0,1692980209,1692982009
2,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991,unl,reference,8,6,12,4,8,2,0.25,1.0,1692982223,1692984023
3,153,2023-08-25 17:23:57.7272734,2023-08-25 17:53:57.8279711,unl,interval,8,6,12,4,8,2,0.25,0.5,1692984237,1692986037
4,154,2023-08-25 17:57:30.6350173,2023-08-25 18:27:30.7363305,unl,interval,8,6,12,4,8,2,0.25,0.5,1692986250,1692988050


In [5]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [6]:
data = pd.read_csv('./exp_filtered.csv', header=0,  index_col=0)
exps = pd.DataFrame(data)

exps.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix,count
2,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991,unl,reference,8,6,12,4,8,2,0.25,1.0,1692982223,1692984023,17416
3,153,2023-08-25 17:23:57.7272734,2023-08-25 17:53:57.8279711,unl,interval,8,6,12,4,8,2,0.25,0.5,1692984237,1692986037,75673
4,154,2023-08-25 17:57:30.6350173,2023-08-25 18:27:30.7363305,unl,interval,8,6,12,4,8,2,0.25,0.5,1692986250,1692988050,153137
5,155,2023-08-25 18:31:03.6332371,2023-08-25 19:01:03.6935448,unl,interval,8,6,12,4,8,2,0.25,0.5,1692988263,1692990063,125112
6,156,2023-08-25 19:04:36.9266682,2023-08-25 19:34:36.9337235,unl,interval,8,6,12,4,8,2,0.25,30.0,1692990276,1692992076,215731
7,157,2023-08-25 19:38:10.1780791,2023-08-25 20:08:10.1796217,unl,interval,8,6,12,4,8,2,0.25,30.0,1692992290,1692994090,274279
8,158,2023-08-25 20:11:43.6270210,2023-08-25 20:41:43.6285805,unl,interval,8,6,12,4,8,2,0.25,30.0,1692994303,1692996103,232539
9,159,2023-08-25 20:45:16.8407778,2023-08-25 21:15:16.8427550,unl,interval,8,6,12,4,8,2,0.25,3.0,1692996316,1692998116,172720
10,160,2023-08-25 21:18:50.1678891,2023-08-25 21:48:50.1692797,unl,interval,8,6,12,4,8,2,0.25,3.0,1692998330,1693000130,195350
11,161,2023-08-25 21:52:23.8902604,2023-08-25 22:22:23.8915283,unl,interval,8,6,12,4,8,2,0.25,3.0,1693000343,1693002143,171780


In [7]:
data = pd.read_csv('./propagation.csv', header=0,  index_col=0)
final = pd.DataFrame(data)

final.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,bandwidth,propagationTime,stddev_propagationTime
0,2,unl,8,6,12,4,8,2,0.25,1.0,23,349.473952,1451.646409,1050.668381
1,3,unl,8,6,12,4,8,2,0.25,0.5,23,413.648073,986.740298,20.491311
2,6,unl,8,6,12,4,8,2,0.25,30.0,23,1056.993541,1325.467710,93.967445
3,9,unl,8,6,12,4,8,2,0.25,3.0,23,744.672331,1294.990615,294.123534
4,12,unl,24,6,12,4,8,2,0.25,1.0,23,824.463965,1315.527934,262.677875
5,15,unl,12,6,12,4,8,2,0.25,1.0,23,961.139534,1524.225677,51.515811
6,18,unl,6,6,12,4,8,2,0.25,1.0,23,621.270477,3237.055361,2525.238950
7,21,unl,8,2,12,4,8,2,0.25,1.0,23,604.560808,1788.039572,306.026796
8,24,unl,8,3,12,4,8,2,0.25,1.0,23,360.859540,1597.645548,326.575399
9,27,unl,8,8,12,4,8,2,0.25,1.0,23,287.544853,1157.719611,261.649231


In [8]:
duplicateMessage = pd.DataFrame()

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_mess = from_influx(url, token, org, "duplicateMessage", exec["startUnix"], exec["endUnix"],"_measurement")
    
        query_mess["identifier"] = row["identifier"]

        query_mess["experiment"] = exec["experiment"]

        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
    
        duplicateMessage = pd.concat([query_mess, duplicateMessage])

        del query_mess

        gc.collect()

duplicateMessage.head(10)


,_time,_measurement,identifier,experiment
0,2023-08-30 02:46:59.183679+00:00,duplicateMessage,144,295
1,2023-08-30 02:46:59.183889+00:00,duplicateMessage,144,295
2,2023-08-30 02:46:59.183984+00:00,duplicateMessage,144,295
3,2023-08-30 02:46:59.183994+00:00,duplicateMessage,144,295
4,2023-08-30 02:46:59.184139+00:00,duplicateMessage,144,295
5,2023-08-30 02:46:59.184150+00:00,duplicateMessage,144,295
6,2023-08-30 02:46:59.184261+00:00,duplicateMessage,144,295
7,2023-08-30 02:46:59.184441+00:00,duplicateMessage,144,295
8,2023-08-30 02:46:59.184486+00:00,duplicateMessage,144,295
9,2023-08-30 02:46:59.184585+00:00,duplicateMessage,144,295


In [18]:
df = duplicateMessage.drop(columns=['_measurement'])

avgPropExp = df.groupby(['experiment', 'identifier']).agg({'_time':['count']})
avgPropExp.columns = avgPropExp.columns.droplevel(0)
avgPropExp.reset_index(inplace=True)
avgPropExp = avgPropExp.drop(columns=['experiment'])
avgPropExp.head(10)

avgOve = avgPropExp.groupby(['identifier']).agg({'count':['mean','std']}).fillna(0)
avgOve.columns = avgOve.columns.droplevel(0)
avgOve.reset_index(inplace=True)

avgOve.head(10)


,identifier,mean,std
0,2,4.369040e+05,591902.115969
1,3,5.217123e+05,207294.360093
2,6,1.412220e+06,221307.397559
3,9,9.942037e+05,90174.581792
4,12,1.086024e+06,657810.889190
5,15,1.201043e+06,208678.232348
6,18,7.683017e+05,65413.322568
7,21,7.785740e+05,500522.442187
8,24,4.573593e+05,344040.074482
9,27,3.345777e+05,360148.453783


In [19]:
finalOve = final.merge(avgOve, on='identifier')
finalOve = finalOve.rename(columns={'mean' : 'messageOverhead', 'std' : 'stddev_messageOverhead'})#.drop(columns=['std'])

finalOve.to_csv('overhead.csv')
finalOve.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,bandwidth,propagationTime,stddev_propagationTime,messageOverhead,stddev_messageOverhead
0,2,unl,8,6,12,4,8,2,0.25,1.0,23,349.473952,1451.646409,1050.668381,4.369040e+05,591902.115969
1,3,unl,8,6,12,4,8,2,0.25,0.5,23,413.648073,986.740298,20.491311,5.217123e+05,207294.360093
2,6,unl,8,6,12,4,8,2,0.25,30.0,23,1056.993541,1325.467710,93.967445,1.412220e+06,221307.397559
3,9,unl,8,6,12,4,8,2,0.25,3.0,23,744.672331,1294.990615,294.123534,9.942037e+05,90174.581792
4,12,unl,24,6,12,4,8,2,0.25,1.0,23,824.463965,1315.527934,262.677875,1.086024e+06,657810.889190
5,15,unl,12,6,12,4,8,2,0.25,1.0,23,961.139534,1524.225677,51.515811,1.201043e+06,208678.232348
6,18,unl,6,6,12,4,8,2,0.25,1.0,23,621.270477,3237.055361,2525.238950,7.683017e+05,65413.322568
7,21,unl,8,2,12,4,8,2,0.25,1.0,23,604.560808,1788.039572,306.026796,7.785740e+05,500522.442187
8,24,unl,8,3,12,4,8,2,0.25,1.0,23,360.859540,1597.645548,326.575399,4.573593e+05,344040.074482
9,27,unl,8,8,12,4,8,2,0.25,1.0,23,287.544853,1157.719611,261.649231,3.345777e+05,360148.453783
